# Frank-Wolfe Algorithm

*Mar, 2017 by K.Wu*

In this report, network are presented as a CSV file:

- [**/data/Frank-Wolfe-algo-data.csv**](https://github.com/wklchris/Reports/blob/master/data/Frank-Wolfe-algo-data.csv): All links of the network.

First, here are the packages needed in this report. Install: 

- Python 3
- numpy package
- pandas package

to continue.

In [1]:
import os
import numpy as np
import pandas as pd

## Introduction

In this report, we have a network like below:

![Network Picture](https://github.com/wklchris/Reports/blob/master/data/Frank-Wolfe-algo-network.png)

The free flow travel time $t_a$ and capacities $c_a$ for each type of link are:

- Black links: $t_a = 10, c_a = 50$
- Red links: $t_a = 30, c_a = 200$
- Yellow links: $t_a = 15, c_a = 200$
- Cyan links: $t_a = 0, c_a = 2000$

We also have the O-D table of this network, shown as below:

In [2]:
ODarr = pd.DataFrame([[100, 200, 300, 50],
                      [100, 200, 200, 100],
                      [300, 50, 100, 100],
                      [200, 500, 50, 200]])

# Rename columns & rows
ODarr.columns = list(range(201, 205))
ODarr.index = list(range(101, 105))

ODarr

,201,202,203,204
101,100,200,300,50
102,100,200,200,100
103,300,50,100,100
104,200,500,50,200


Notice that the capacity of cyan links are 2000, which is larger than demand of any single destination node ($201\sim 204$). 

**So we can exclude those O/D nodes in our network representation step.** We only need to add them to the results in the very last step --- and their costs are also constant when the demand of each destination is fixed.

## Network Representation



In [3]:
linkarr = pd.read_csv(r"{}/data/Frank-Wolfe-algo-data.csv".format(os.getcwd()))
linkarr.head()

,begin,end,type
0,1,2,black
1,2,3,black
2,3,4,black
3,14,15,black
4,15,16,black


As for the costs, we use BPR function to compute the cost on each link. Cost on link $a$, called $t_a$, is a function of $x_a$:

$$ t_a(x_a) = t_a^0 \left[1 + \alpha\left(\frac{x_a}{c_a'}\right)\right] $$

where $\alpha = 0.15, \beta = 4, c_a' = 0.9c_a$.